# **Aula 4: Respondendo perguntas**

## **Selecionando o modelo**

Modelo: [roberta-base for QA](https://huggingface.co/deepset/roberta-base-squad2)

In [ ]:
import os
import torch
from transformers import pipeline

if torch.backends.mps.is_available():
    os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
    os.environ["USE_MPS"] = "1"
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    os.environ["PYTORCH_MPS_DEVICE_OVERRIDE"] = "1"
    device = torch.device('mps')
    torch.set_default_device(device)
    print(f"Using device: {device}")
else:
    device = torch.device('cpu')
    torch.set_default_device(device)
    print(f"Using device: {device}")

In [ ]:
QA_modelo = pipeline('question-answering', model='deepset/roberta-base-squad2', device=device)   

In [ ]:
def obter_resposta(pergunta, contexto):
  return QA_modelo(question=pergunta, context=contexto)

In [ ]:
pergunta = 'What are the payment options available?'
contexto = 'In our e-commerce, you can make payments using credit card, debit card, Apple Pay, or through bank transfers.'

resposta = obter_resposta(pergunta, contexto)

In [ ]:
resposta

## **Aplicando o modelo aos dados**

In [ ]:
contextos = {
    "How do I create an account?": "You can create an account by clicking on the 'Sign Up' button on our homepage. You will need to provide your email address, create a password, and fill in some basic information about yourself. An account will help you track your orders, manage your personal settings, and speed up future transactions.",
    "Which payment methods do you accept?": "We accept a wide range of payment methods including Visa, MasterCard, American Express, Discover, PayPal, and Apple Pay. You can also pay using store credit or gift cards issued by our company.",
    "How can I track my order?": "Once your order has shipped, you will receive an email with a tracking number. You can use this number on our website's tracking page to see the current status of your delivery.",
    "Do you offer international shipping?": "Yes, we offer international shipping to most countries. Shipping costs and delivery times vary depending on the destination. All applicable customs fees, taxes, and duties are the responsibility of the customer and are calculated at checkout.",
    "How long does delivery take?": "For standard shipping, deliveries typically take between 3 to 5 business days. For expedited shipping, expect your order to arrive within 1 to 2 business days. Delivery times may vary based on your location and the time of the year.",
    "What is your return policy?": "Our return policy allows you to return products within 30 days of receiving them. Items must be in their original condition and packaging. Some items, such as perishable goods, are not eligible for return.",
    "Can I change or cancel my order after it's been placed?": "You can change or cancel your order within 24 hours of placing it without any additional charge.To make changes or cancel your order, please contact our customer service immediately.",
    "What should I do if I receive a damaged item?": "If you receive a damaged item, please contact our customer service within 48 hours of delivery to report the damage. You will need to provide your order number, the description of the damage, and photographic evidence. We will arrange for a replacement or refund as appropriate.",
    "How do I reset my password?": "If you've forgotten your password, go to the login page and click on 'Forgot Password'. Enter your email address and we will send you a link to reset your password. For security purposes, this link will expire within 24 hours."
}

In [ ]:
def respondendo_faq(pergunta):
  contexto = contextos[pergunta]
  resultado = QA_modelo(question=pergunta, context=contexto)
  return resultado['answer']

In [ ]:
respondendo_faq('How do I create an account?')

In [ ]:
respondendo_faq('Which payment methods do you accept?')

In [ ]:
respondendo_faq('Do you offer international shipping?')

In [ ]:
respondendo_faq("Can I change or cancel my order after it's been placed?")

## **Criando uma interface**

In [ ]:
# pip install gradio

In [ ]:
import gradio as gr

app = gr.Interface(
    fn=respondendo_faq,
    inputs=gr.Dropdown(choices=list(contextos.keys()), label='Select your question'),
    outputs='text',
    title='E-commerce FAQ',
    description='Select a question to get an answer from our FAQ'
)

app.launch(share=False)